In [ ]:
%matplotlib inline

import suPAErnova as snpae
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Some useful paths to have access to
cwd = Path.cwd()
examples_dir = cwd.parent.parent.parent
data_dir = examples_dir / "suPAErnova_data" # Put your data into this directory

# Global Configuration

In [ ]:
verbose = False # Increase log verbosity
force = False # Don't rerun steps if not necessary

cfg = snpae.setup_global_config({}, verbose=verbose, force=force) # Just pass an empty dictionary to initialise

# Data Configuration
If we just tried to run `DATA` without providing the required options, then `SuPAErnova` will fail, providing a (hopefully) useful description of what's missing.

In [ ]:
cfg["DATA"] = {}
data = snpae.steps.DATAStep(cfg)

Let's instead actually provide proper configuration options

In [ ]:
# Note that these keys *MUST* be captilalised. This is not the case when using a `suPAErnova.toml` config file.
data_cfg = {
    # === Required Keys ===

    # Path to directory containing data.
    #   Can be absolute or relative to the base path.
    "DATA_DIR": str(data_dir), # Needs to be a string so that SuPAErnova can validate it
    
    # Metadata CSV containing SN names and SALT fit parameters.
    #   Can be absolute or relative to the data path.
    "META": "meta.csv",

    # TXT file containing additional SALT fit parameters.
    #   Can be absolute or relative to the data path.
    "IDR": "IDR_eTmax.txt",

    # TXT file containing a mask of bad spectra / wavelength ranges.
    #   Can be absolute or relative to the data path.
    "MASK": "mask_info_wmin_wmax.txt",

    # === Optional Keys ===

    # Which assumed cosmology to use when running SALT models.
    #   Available cosmological can be found [here](https://docs.astropy.org/en/stable/cosmology/realizations.html)
    #   Defaults to WMAP7
    "COSMOLOGICAL_MODEL": "WMAP7",

    # The absolute path to an existing SALT2/3 model, or the name of an existing SNCosmo SALT2/3 model.
    #   Defaults to salt3
    "SALT_MODEL": "salt3",

    # Minimum phase for spectral data, relative to peak. Spectral data earlier than this phase will be cut.
    #   Defaults to -10.0
    "MIN_PHASE": -10,

    # Maximum phase for spectral data, relative to peak. Spectral data later than this phase will be cut.
    #   Defaults to 40.0
    "MAX_PHASE": 40,

    # The fraction of data to be used for training, with the rest of the data going to testing and validation.
    #   Defaults to 0.75
    "TRAIN_FRAC": 0.75,

    # The seed used throughout data preperation, in particular for randomly splitting the data into training, testing, and validation bins.
    #   Defaults to 12345
    "SEED": 12345,

    # === Analysis Keys ===
    "ANALYSIS": {
        # Which spectra to plot
        #    str: The name of a single spectrum to plot
        #    list[str]: Names of each spectrum to plot
        #    True: Plot every spectrum
        "PLOT_SPECTRA": True
    }
}

cfg["DATA"] = data_cfg
data = snpae.steps.DATAStep(cfg)
print(data)

# Setup
We can now run the first stage of the `DATA` step, setup. Most `SuPAErnova` functions return an object called a `RequirementReturn[T]`, which is a two-tuple whose first element is `True` if the function ran successfully, and `False` otherwise. The second element is either the result of that function (of type `T`), or a `str` describing why the function failed.

In [ ]:
success, result = data.setup()
if not success: # Make sure you handle failures appropriately!
    data.log.error(f"Error running setup: {result}")
print(data)

# Run
Now we can run the `DATA` step, loading in the data, transforming it into something useful, and splitting the data into training and tesing subsamples.

In [ ]:
success, result = data.run()
if not success: # Make sure you handle failures appropriately!
    data.log.error(f"Error running: {result}")
print(data)

# Result
Finally, we run the `DATA` result function, to save the results of the run function to the output directory.

In [ ]:
success, result = data.result()
if not success: # Make sure you handle failures appropriately!
    data.log.error(f"Error saving results: {result}")
print(data)

# Analysis
Though you can obviously write your own plotting and analysis functions, `SuPAErnova` provides a number of pre-made analysis functions you can use. If defined in your `data_cfg` dictionary, thesis analysis functions can be called via the `analyse` function.

In [ ]:
success, result = data.analyse()
if not success: # Make sure you handle failures appropriately!
    data.log.error(f"Error analysing: {result}")

img = mpimg.imread(data.plotpath / "CSS110918_01.png")
imgplot = plt.imshow(img)
plt.show()